In [1]:
#import necessary libraries and frameworks
import gdspy
import pandas as pd
import numpy as np
import sys

import matplotlib.pylab as plt
import matplotlib.pyplot

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from numpy import savetxt
from PIL import Image

In [2]:
#input parameters
baseUnit = 1 #layout scale(microns)
m=baseUnit*1e6 
nm = m*1E-9
um = m*1E-6
mm = m*1e-3
# scaling = 5; #scaling factor

periodX = 3*um #period along x-axis
periodY = 3*um #period along y-axis
Xmax = 1.5*mm #half of the metasurface length
Ymax = 1.5*mm #half of the metasurface width
Nx = 2*Xmax/periodX #number of meta-atoms along x-axis
Ny = 2*Ymax/periodY #number of meta-atoms along y-axis
phaseStep = 90 #phase step in degrees

In [3]:
#read phase map
# phaseMap = pd.read_csv("a-state_ideal.txt", header=None)
# phaseMap = phaseMap.values.tolist()

In [4]:
#generating analytical phase map
f = 2*mm #focal distance
lam = 5.2*um #wavelength

In [5]:
#example of 1D ideal planar lens
x = np.linspace(-Xmax,Xmax,Nx)
phase_analyt0 = 2*np.pi/lam*(np.sqrt(x**2+f**2)-f)
phase_analyt = np.remainder(phase_analyt0,2*np.pi)

#plotting the phase dependence
plt.plot(x, phase_analyt)
plt.xlabel('x, um')
plt.ylabel('phase, rad')
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(20, 4)

TypeError: object of type <class 'float'> cannot be safely interpreted as an integer.

In [6]:
#generating 2D phase map for an ideal planar lens
x = np.linspace(-Xmax,Xmax,Nx)
y = np.linspace(-Ymax,Ymax,Ny)
phase = []

X,Y = np.meshgrid(x,y)
phaseMap = np.rint(180/np.pi*np.remainder(2*np.pi/lam*(np.sqrt(X**2+Y**2+f**2)-f),2*np.pi)) #in degrees  

TypeError: object of type <class 'float'> cannot be safely interpreted as an integer.

In [7]:
savetxt('metasurface_phaseMap_cState.csv', phaseMap, delimiter=',')


NameError: name 'phaseMap' is not defined

In [8]:
#2D surface plot
fig = plt.figure()
plt.contourf(X,Y,phaseMap, cmap=cm.jet)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(10, 10)

NameError: name 'X' is not defined

<Figure size 432x288 with 0 Axes>

In [9]:
#function to clean up cells
def clearCell(cell):
    cell.remove_polygons(lambda pts,layer,datatype:True)

In [10]:
#layer and datatype tags
# ld_Si = {"layer":1, "datatype":1}

In [11]:
#creating a gdspy library
lib = gdspy.GdsLibrary()

In [12]:
#function to convert bmp file to gds polygon
def bWToGDS(Width, Height, bwArray, layer, datatype, matVal=0):
    #The Black Pixels will be represented as material, white is air
    YRes=len(bwArray)
    XRes=len(bwArray[0])
    pX=Width/XRes
    pY=Height/YRes
    rects=[]
    for i in range(len(bwArray)):
        for j in range(len(bwArray[0])):
           # print(bwArray[i][j])
            if bwArray[i][j]==matVal:
               # print("Adding Rect")
                #print((pX*i, pY*j))
                rects.append(gdspy.Rectangle((pX*j,-pY*i),(pX*(j+1), -pY*(i+1)), layer, datatype))
    patt=gdspy.boolean(rects[0], rects[1:], "or", max_points=0)
    
    return patt

In [13]:
# loading the bmp files
bmp_path = 'C:/Users/Misha Shalaginov/Desktop/Python codes/metasurface/free-shape unit cells'
file_name1 = bmp_path + '/meta-atom1'+'.bmp'
file_name2 = bmp_path + '/meta-atom2'+'.bmp'
file_name3 = bmp_path + '/meta-atom3'+'.bmp'
file_name4 = bmp_path + '/meta-atom4'+'.bmp'

bmp1 = Image.open(file_name1)
MA1 = np.asarray(bmp1).copy() 

bmp2 = Image.open(file_name2)
MA2 = np.asarray(bmp2).copy() 

bmp3 = Image.open(file_name3)
MA3 = np.asarray(bmp3).copy() 

bmp4 = Image.open(file_name4)
MA4 = np.asarray(bmp4).copy() 

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Misha Shalaginov/Desktop/Python codes/metasurface/free-shape unit cells/meta-atom1.bmp'

In [14]:
# bwImg = Image.fromarray(MA1)

In [15]:
#test of bWToGDS
MA1poly = bWToGDS (periodX, periodY, MA1, 1, 1, matVal=255)
MA2poly = bWToGDS (periodX, periodY, MA2, 1, 1, matVal=255)
MA3poly = bWToGDS (periodX, periodY, MA3, 1, 1, matVal=255)
MA4poly = bWToGDS (periodX, periodY, MA4, 1, 1, matVal=255)

NameError: name 'MA1' is not defined

In [16]:
# #MAPPING PHASE TO META-ATOM RADIUS
# # phaseToRadiusMap = [[45+45*i,nm*(50+10*i)]for i in range(8)]
# # phase to radius relations are taken from numerical simulations, vis WFOV a-Si metalens @ 680nm, height 800nm
# phaseToRadius = [[0, 93*scaling*nm],[45, 87*scaling*nm], [90, 81.5*scaling*nm], [135,78*scaling*nm], [180,75*scaling*nm], [225, 59*scaling*nm], [270, 54.5*scaling*nm], [315, 46*scaling*nm]]
# metaatomRadius = [93*scaling*nm, 87*scaling*nm, 81.5*scaling*nm, 78*scaling*nm, 75*scaling*nm, 59*scaling*nm, 54.5*scaling*nm, 46*scaling*nm]
# phaseValue = [0, 45, 90, 135, 180, 225, 270, 315]
phaseValue = [0, 90, 180, 270]

In [17]:
# #GENERATING META-ATOM lIBRARY
# # disks = []
# # for i in range(len(metaatomRadius)):
# # disks.append(gdspy.Round((periodX*i, 0), metaatomRadius[i], number_of_points=12))  
# disk1 = gdspy.Round((0, 0), metaatomRadius[0], tolerance=0.001)
# disk2 = gdspy.Round((0, 0), metaatomRadius[1], tolerance=0.001)
# disk3 = gdspy.Round((0, 0), metaatomRadius[2], tolerance=0.001)
# disk4 = gdspy.Round((0, 0), metaatomRadius[3], tolerance=0.001)
# disk5 = gdspy.Round((0, 0), metaatomRadius[4], tolerance=0.001)
# disk6 = gdspy.Round((0, 0), metaatomRadius[5], tolerance=0.001)
# disk7 = gdspy.Round((0, 0), metaatomRadius[6], tolerance=0.001)
# disk8 = gdspy.Round((0, 0), metaatomRadius[7], tolerance=0.001)

In [18]:
MA1cell=lib.new_cell("metaatom-1")
MA1cell.add(MA1poly)

MA2cell=lib.new_cell("metaatom-2")
MA2cell.add(MA2poly)

MA3cell=lib.new_cell("metaatom-3")
MA3cell.add(MA3poly)

MA4cell=lib.new_cell("metaatom-4")
MA4cell.add(MA4poly)

# MA5cell=lib.new_cell("metaatom-5")
# MA5cell.add(disk5)

# MA6cell=lib.new_cell("metaatom-6")
# MA6cell.add(disk6)

# MA7cell=lib.new_cell("metaatom-7")
# MA7cell.add(disk7)

# MA8cell=lib.new_cell("metaatom-8")
# MA8cell.add(disk8)

MAcell = [MA1cell, MA2cell, MA3cell, MA4cell]
# MAcell = [MA1cell, MA2cell, MA3cell, MA4cell, MA5cell, MA6cell, MA7cell, MA8cell]

# lib.add([MA1cell, MA2cell, MA3cell,MA4cell,MA5cell,MA6cell,MA7cell,MA8cell])
# lib.write_gds('Metasurface1.gds')
# gdspy.LayoutViewer()

NameError: name 'MA1poly' is not defined

In [19]:
#CONVERTING PHASE MAP to RADIUS MAP
def phaseToCellMap(center, phase, phaseValue, MAcell):
    for i in range(len(phaseValue)):
        if phase<phaseValue[i]+phaseStep/2 and phase>=phaseValue[i]-phaseStep/2:
            return gdspy.CellReference(MAcell[i], center)
        elif phase>=max(phaseValue)+phaseStep/2:
            return gdspy.CellReference(MAcell[0], center)

In [20]:
#DISCRETIZING PHASE MAP
def phaseToCellMap(center, phase, phaseValue, MAcell):
    for i in range(len(phaseValue)):
        if phase<phaseValue[i]+phaseStep/2 and phase>=phaseValue[i]-phaseStep/2:
            phaseMap[i]
        elif phase>=max(phaseValue)+phaseStep/2:
            return gdspy.CellReference(MAcell[0], center)

In [21]:
#GENERATING SPATIAL X,Y GRID
def spatialMapGenerator(periodX, periodY, phaseMap):
    spatialMap = [[[periodX*x, periodY*y] for x in range(len(phaseMap))] for y in range(len(phaseMap[0]))]
    return spatialMap

In [22]:
#PUTTING IDEAL PHASE MAP ON A GRID (discretization)
spatialMap = spatialMapGenerator(periodX, periodY, phaseMap)
spatialMap

NameError: name 'phaseMap' is not defined

In [23]:
#CREATING THE LIBRARY of META-ATOMS
#function to generate a circle of radius 'r' at position 'center'
# def phaseDiskPoly(phase, phaseToRadiusMap, center):
#     r = phaseToDiskRadius(phase, phaseToRadiusMap)
#     disk = gdspy.Round(center,r) #!change round; return the cell reference, #documention on self reference
#     return disk

In [24]:
#function to generate 2D map of circles (metasurface)
def metalensGenerator(phaseMap, phaseToCellMap, spatialMap):
    metasurface = []
    for x in range(len(phaseMap)):
        for y in range(len(phaseMap[0])):
            metasurface.append(phaseToCellMap(spatialMap[x][y], phaseMap[x][y], phaseValue, MAcell))
    return metasurface
        

In [25]:
metalens = metalensGenerator(phaseMap, phaseToCellMap, spatialMap)

NameError: name 'phaseMap' is not defined

In [26]:
metalens

NameError: name 'metalens' is not defined

In [27]:
gdspy.current_library = gdspy.GdsLibrary()


In [28]:
Lens=gdspy.Cell("Lens-1")
Lens.add(metalens)

NameError: name 'metalens' is not defined

In [29]:
#writing gds file
gdspy.current_library = gdspy.GdsLibrary()
gdspy.current_library.add(Lens)
# gdspy.current_library.add(top)
# top.add(gdspy.CellReference(Lens))
# top.add(gdspy.CellReference(Lens, (2*mm,2*mm)))
gdspy.write_gds('20200830_metasurface3mm.gds')